In [3]:
#import dependencies

import numpy as np
import pandas as pd
import pickle 

In [4]:
movies = pd.read_csv(r"C:\Users\shubh\Desktop\movie recommended system\tmdb_5000_movies.csv")
credit = pd.read_csv(r"C:\Users\shubh\Desktop\movie recommended system\tmdb_5000_credits.csv")

In [5]:
movies = movies.merge(credit,on='title')

In [6]:
#importaint columns which can usefull for creating tag
#movie id
#genres
#keywords
#overview
#title
#cast
#crew
movies = movies[['movie_id','title','genres','keywords','overview','cast','crew']]

In [7]:
movies.dropna(inplace=True)

In [8]:
#movies.columns

In [9]:
#movies

In [10]:
#movies.iloc[0].genres

In [11]:
import ast
def convert(obj):
    l=[]
    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l
        

In [12]:
movies['genres'] = movies['genres'].apply(convert)

In [13]:
#movies['genres']

In [14]:
movies['keywords'] = movies['keywords'].apply(convert)

In [15]:
#movies['keywords'] 

In [16]:
#movies['cast'].value_counts()

In [17]:
def convert3(obj):
    l=[]
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            l.append(i['name'])
            counter+=1
        else:
           break

    return l

In [18]:
movies['cast']  = movies['cast'].apply(convert3)

In [19]:
#movies['cast']

In [20]:
#movies['crew'].value_counts()

In [21]:
def crew(obj):
    l=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            l.append(i['name'])
            break
    return l


In [22]:
movies['crew'] = movies['crew'].apply(crew)  

In [23]:
#movies['crew'].value_counts()

In [24]:
movies['overview'] = movies['overview'].apply(lambda x: x.split())

In [25]:
#movies['overview'].value_counts()

In [26]:
movies['cast'] = movies['cast'].apply(lambda x : [i.replace(' ','') for i in x])
movies['crew'] = movies['crew'].apply(lambda x: [i.replace(' ','') for i in x])
movies['genres'] = movies['genres'].apply(lambda x: [i.replace(' ','') for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x: [i.replace(' ','') for i in x])

In [27]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [28]:
#movies['tags'].value_counts()

In [29]:
new_df = movies[['movie_id','title','tags']]

In [30]:
#new_df

In [31]:
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))

C:\Users\shubh\AppData\Local\Temp\ipykernel_9696\1824047427.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))


In [32]:
#new_df['tags'].value_counts()

In [33]:
new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())

C:\Users\shubh\AppData\Local\Temp\ipykernel_9696\1380776331.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())


In [34]:
#new_df['tags'].value_counts()

In [35]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [36]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [37]:
new_df['tags'] = new_df['tags'].apply(stem)

C:\Users\shubh\AppData\Local\Temp\ipykernel_9696\3213734980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


In [38]:
#new_df

In [39]:
from sklearn.feature_extraction.text import CountVectorizer

In [40]:
cv = CountVectorizer(max_features=5000, stop_words='english')

In [41]:
vector=cv.fit_transform(new_df['tags']).toarray()

In [42]:
#vector

In [43]:
from sklearn.metrics.pairwise import cosine_similarity

In [44]:
similarity = cosine_similarity(vector)

In [45]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:10]

[(1214, 0.28676966733820225),
 (2405, 0.26901379342448517),
 (3728, 0.2605130246476754),
 (507, 0.255608593705383),
 (539, 0.2503866978335957),
 (582, 0.24511108480187255),
 (1202, 0.24455799402225925),
 (1192, 0.2367785320221084),
 (61, 0.23179316248638274)]

In [46]:
def recommend(movie):
    movie_index = new_df[new_df['title']==movie].index[0]
    distance = similarity[movie_index]
    movie_list = sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:10]
    for i in movie_list:
        print(new_df.iloc[i[0]].title)

In [47]:
recommend('Independence Day')

Aliens vs Predator: Requiem
Aliens
Falcon Rising
Independence Day
Titan A.E.
Battle: Los Angeles
Predators
Small Soldiers
Jupiter Ascending


In [48]:
# data transfer on software interface
pickle.dump(new_df,open('movies.pkl','wb'))


In [49]:
#i dont't know how to direct data file to pickle 
pickle.dump(new_df.to_dict(),open('movie_dict.pkl','wb'))

In [50]:
pickle.dump(similarity,open('similarity.pkl','wb'))